<a href="https://colab.research.google.com/github/ricotta-jpgomes/estante_virtual/blob/main/GCP/GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catálogo de preços para VMs na GCP e suas configurações
---

**Autor**: João Paulo Gomes Ricotta

**Ano**: 2025





> Este notebook consolida uma base de dados com preços e configurações de diferentes instâncias de VMs disponíveis na Google Cloud Platform (GCP), e faz parte do trabalho de conclusão de curso para o MBA em Data Science e Analytics da USP-ESALQ. Os dados do catálogo são públicos, e podem ser consultados pela api de preços de varejo da Azure. As configurações de cada máquina, por sua vez, são obtidas pela api de gerenciamento de conta, sendo necessário a criação de uma conta e um usuário impessoal (aplicativo) dentro do provedor.



## Configuração do ambiente 💻

In [ ]:
!pip install python-dotenv

In [ ]:
import json
from datetime import datetime, timezone, timedelta  # Manipulação de datas

import numpy as np
import pandas as pd
import pytz  # Manipulação de fuso horário
import requests
from dotenv import find_dotenv, load_dotenv, set_key
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
# --- Bloco de Configuração ---
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/GCP/gcp_credentials.json'
SCOPES = ['https://www.googleapis.com/auth/compute.readonly', 'https://www.googleapis.com/auth/cloud-billing', 'https://www.googleapis.com/auth/cloud-platform']
PROJECT_ID = 'friendly-cargo-463721-c0'

In [ ]:
# Encontra o arquivo .env no caminho especificado
# Se não encontrar, ele pode criar um arquivo .env no diretório especificado
env_file = find_dotenv("/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/.env")
if not env_file:
    env_file = '/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/.env'  # Cria o arquivo .env no diretório atual se não existir

In [ ]:
diff = timedelta(hours = -3)
tzone = timezone(diff)
extraction_date = datetime.now(tz=tzone) # obtendo a data e hora da extração dos dados
extraction_date = extraction_date.astimezone(pytz.timezone("America/Sao_Paulo"))

set_key(env_file, 'EXTRACTION_DATE', f'{extraction_date.strftime("%d-%m-%Y")}')

(True, 'EXTRACTION_DATE', '06-07-2025')

## Coleta dos Dados 🎣

### Configuração das credenciais de autenticação

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )

In [ ]:
# Constrói um "cliente" para interagir com a API do Compute Engine
compute_service = build('compute', 'v1', credentials=credentials)

### Obtendo os detalhes de configuração das máquinas

#### Busca por zonas

In [ ]:
# --- Autenticação e Coleta (Zonas disponíveis para consulta) ---
try:

    print(f"Buscando zonas disponíveis para o projeto {PROJECT_ID}...")

    # Faz a chamada à API para listar as regiões
    request = compute_service.zones().list(project=PROJECT_ID)
    response = request.execute()

    # Extrai o nome de cada região da resposta e adiciona a uma lista
    zones = [zone['name'] for zone in response.get('items', [])]

    print("\n-> Busca de regiões concluída.")

    # Imprime a lista Python final
    print("-> Lista de zonas do Google Cloud Platform:")
    print(zones)

    print(f"\nTotal de {len(zones)}  zonas encontradas.")


except FileNotFoundError:
    print(f"ERRO: Arquivo de credenciais '{SERVICE_ACCOUNT_FILE}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
    print("--> Possível causa: A 'Compute Engine API' não está ativada no seu projeto ou a conta de serviço não tem o papel de 'Leitor' (Viewer) no projeto.")

Buscando zonas disponíveis para o projeto friendly-cargo-463721-c0...

-> Busca de regiões concluída.
-> Lista de zonas do Google Cloud Platform:
['us-east1-b', 'us-east1-c', 'us-east1-d', 'us-east4-c', 'us-east4-b', 'us-east4-a', 'us-central1-c', 'us-central1-a', 'us-central1-f', 'us-central1-b', 'us-west1-b', 'us-west1-c', 'us-west1-a', 'europe-west4-a', 'europe-west4-b', 'europe-west4-c', 'europe-west1-b', 'europe-west1-d', 'europe-west1-c', 'europe-west3-c', 'europe-west3-a', 'europe-west3-b', 'europe-west2-c', 'europe-west2-b', 'europe-west2-a', 'asia-east1-b', 'asia-east1-a', 'asia-east1-c', 'asia-southeast1-b', 'asia-southeast1-a', 'asia-southeast1-c', 'asia-northeast1-b', 'asia-northeast1-c', 'asia-northeast1-a', 'asia-south1-c', 'asia-south1-b', 'asia-south1-a', 'australia-southeast1-b', 'australia-southeast1-c', 'australia-southeast1-a', 'southamerica-east1-b', 'southamerica-east1-c', 'southamerica-east1-a', 'africa-south1-a', 'africa-south1-b', 'africa-south1-c', 'asia-east2

#### Busca por máquinas

Com a lista de zonas disponíveis devidamente configurada, vamos buscar as configurações de cada máquina (essa busca é feita por zonas).

In [ ]:
machine_specs = []
for zone in zones:
      print(f"Buscando especificações de máquinas na zona {zone}...")

      # Faz a chamada à API para listar os tipos de máquina
      try:
        request = compute_service.machineTypes().list(project=PROJECT_ID, zone=zone)

        # O resultado pode vir em páginas, então fazemos um laço
        while request is not None:
            response = request.execute()

            # Para cada máquina na resposta, extraímos os dados que queremos
            if 'items' in response:

                for machine_type in response['items']:
                    discos_temporarios = machine_type.get('ephemeralDisks', [])
                    tamanho_total_temp_gb = sum(int(disk.get('diskGb', 0)) for disk in discos_temporarios)


                    machine_specs.append({
                        'machineType': machine_type.get('name'),
                        'machineFamily': machine_type.get('name', '').split('-')[0].upper(),
                        'numberOfCores': machine_type.get('guestCpus'),
                        'memoryInMiB': machine_type.get('memoryMb'), # A API retorna em MiB
                        'ephemeralDiskGiB': tamanho_total_temp_gb,
                        'zone': zone
                        })

            # Pega o token da próxima página
            request = compute_service.machineTypes().list_next(previous_request=request, previous_response=response)
      except Exception as e:
        print(f"Ocorreu um erro: {e}")
print("Busca de especificações concluída.")

Buscando especificações de máquinas na zona us-east1-b...
Buscando especificações de máquinas na zona us-east1-c...
Buscando especificações de máquinas na zona us-east1-d...
Buscando especificações de máquinas na zona us-east4-c...
Buscando especificações de máquinas na zona us-east4-b...
Buscando especificações de máquinas na zona us-east4-a...
Buscando especificações de máquinas na zona us-central1-c...
Buscando especificações de máquinas na zona us-central1-a...
Buscando especificações de máquinas na zona us-central1-f...
Buscando especificações de máquinas na zona us-central1-b...
Buscando especificações de máquinas na zona us-west1-b...
Buscando especificações de máquinas na zona us-west1-c...
Buscando especificações de máquinas na zona us-west1-a...
Buscando especificações de máquinas na zona europe-west4-a...
Buscando especificações de máquinas na zona europe-west4-b...
Buscando especificações de máquinas na zona europe-west4-c...
Buscando especificações de máquinas na zona euro

In [ ]:
 # Cria o DataFrame final com as especificações
df_gcp_specs = pd.DataFrame(machine_specs)

print("\nAmostra do DataFrame de especificações de VMs do GCP:")
display(df_gcp_specs.head())

print(f"\nTotal de {len(df_gcp_specs)} tipos de máquina encontrados para as zonas disponíveis.")


Amostra do DataFrame de especificações de VMs do GCP:


,machineType,machineFamily,numberOfCores,memoryInMiB,ephemeralDiskGiB,zone
0,a2-highgpu-1g,A2,12,87040,0,us-east1-b
1,a2-highgpu-2g,A2,24,174080,0,us-east1-b
2,a2-highgpu-4g,A2,48,348160,0,us-east1-b
3,a2-highgpu-8g,A2,96,696320,0,us-east1-b
4,a4-highgpu-8g,A4,224,4063232,0,us-east1-b



Total de 25365 tipos de máquina encontrados para as zonas disponíveis.


In [ ]:
df_gcp_specs['regionCode'] = df_gcp_specs['zone'].str.slice(0,-2)
df_gcp_specs.drop(columns='zone', inplace=True)
df_gcp_specs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25365 entries, 0 to 25364
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   machineType       25365 non-null  object
 1   machineFamily     25365 non-null  object
 2   numberOfCores     25365 non-null  int64 
 3   memoryInMiB       25365 non-null  int64 
 4   ephemeralDiskGiB  25365 non-null  int64 
 5   regionCode        25365 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.2+ MB


### Requisição de skus da Google para VMs (custos)

In [ ]:
api_url = "https://cloudbilling.googleapis.com/v1/services/6F81-5844-456A/skus"
try:
    # Gera/Atualiza o token de acesso
    credentials.refresh(Request())
    access_token = credentials.token

    # Monta o cabeçalho de autorização (Bearer Token)
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    table_data = []
    page_token = None
    page_count = 0

    print("Iniciando a coleta de dados do GCP (método manual com Bearer Token)...")

    while True:
        page_count += 1
        params = {}
        if page_token:
            params['pageToken'] = page_token

        print(f"Buscando página {page_count}...")

        # 4. Faz a requisição passando o cabeçalho de autorização
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()

        # ... (o resto do laço de processamento é igual) ...
        json_data = response.json()
        if 'skus' in json_data:
            table_data.extend(json_data['skus'])
        page_token = json_data.get('nextPageToken')
        if not page_token:
            break

    print(f"\n-> Coleta de dados do GCP finalizada. Total de {len(table_data)} SKUs encontradas.")
    df_gcp_skus = pd.DataFrame(table_data)
    df_gcp_skus.info()
    display(df_gcp_skus.head())


except FileNotFoundError:
    print(f"ERRO: Arquivo de credenciais '{SERVICE_ACCOUNT_FILE}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Iniciando a coleta de dados do GCP (método manual com Bearer Token)...
Buscando página 1...
Buscando página 2...
Buscando página 3...
Buscando página 4...
Buscando página 5...
Buscando página 6...

-> Coleta de dados do GCP finalizada. Total de 28079 SKUs encontradas.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28079 entries, 0 to 28078
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 28079 non-null  object
 1   skuId                28079 non-null  object
 2   description          28079 non-null  object
 3   category             28079 non-null  object
 4   serviceRegions       28079 non-null  object
 5   pricingInfo          28079 non-null  object
 6   serviceProviderName  28079 non-null  object
 7   geoTaxonomy          28026 non-null  object
dtypes: object(8)
memory usage: 1.7+ MB


,name,skuId,description,category,serviceRegions,pricingInfo,serviceProviderName,geoTaxonomy
0,services/6F81-5844-456A/skus/0001-FC8F-A9AF,0001-FC8F-A9AF,Spot Preemptible E2 Custom Instance Core runni...,"{'serviceDisplayName': 'Compute Engine', 'reso...",[europe-west9],"[{'summary': '', 'pricingExpression': {'usageU...",Google,"{'type': 'REGIONAL', 'regions': ['europe-west9']}"
1,services/6F81-5844-456A/skus/0006-C9C8-BB6F,0006-C9C8-BB6F,Commitment v1: Memory-optimized Cpu in Phoenix...,"{'serviceDisplayName': 'Compute Engine', 'reso...",[us-west8],"[{'summary': '', 'pricingExpression': {'usageU...",Google,"{'type': 'REGIONAL', 'regions': ['us-west8']}"
2,services/6F81-5844-456A/skus/0007-9388-EF75,0007-9388-EF75,M4Ultramem224 Sole Tenancy Instance Ram runnin...,"{'serviceDisplayName': 'Compute Engine', 'reso...",[northamerica-northeast2],"[{'summary': '', 'pricingExpression': {'usageU...",Google,"{'type': 'REGIONAL', 'regions': ['northamerica..."
3,services/6F81-5844-456A/skus/0008-F633-76AA,0008-F633-76AA,Nvidia L4 GPU attached to Spot Preemptible VMs...,"{'serviceDisplayName': 'Compute Engine', 'reso...",[asia-east2],"[{'summary': '', 'pricingExpression': {'usageU...",Google,"{'type': 'REGIONAL', 'regions': ['asia-east2']}"
4,services/6F81-5844-456A/skus/000F-0B14-D302,000F-0B14-D302,C3 Sole Tenancy Instance Ram running in Turin,"{'serviceDisplayName': 'Compute Engine', 'reso...",[europe-west12],"[{'summary': '', 'pricingExpression': {'usageU...",Google,"{'type': 'REGIONAL', 'regions': ['europe-west1..."


#### Tratamento das informações de SKUs

In [ ]:
# Informações sobre a categoria do serviço
df_gcp_skus['serviceName'] = df_gcp_skus['category'].apply(lambda x: x['serviceDisplayName'])
df_gcp_skus['resourceType'] = df_gcp_skus['category'].apply(lambda x: x['resourceFamily'])
df_gcp_skus['resourceName'] = df_gcp_skus['category'].apply(lambda x: x['resourceGroup'])
df_gcp_skus['usage'] = df_gcp_skus['category'].apply(lambda x: x['usageType']) # On-Demand / Reservation

df_gcp_skus.drop(columns='category', inplace=True)
df_gcp_skus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28079 entries, 0 to 28078
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 28079 non-null  object
 1   skuId                28079 non-null  object
 2   description          28079 non-null  object
 3   serviceRegions       28079 non-null  object
 4   pricingInfo          28079 non-null  object
 5   serviceProviderName  28079 non-null  object
 6   geoTaxonomy          28026 non-null  object
 7   serviceName          28079 non-null  object
 8   resourceType         28079 non-null  object
 9   resourceName         28079 non-null  object
 10  usage                28079 non-null  object
dtypes: object(11)
memory usage: 2.4+ MB


In [ ]:
# Eliminando registros de skus cujo modelo de cobrança não será objeto de análise na presente investigação
df_gcp_skus = df_gcp_skus.loc[
    ~df_gcp_skus['usage'].isin(['Preemptible', 'Commit1Mo', 'CmtCudPremium'])
    ]

df_gcp_skus['usage'].unique()

array(['Commit1Yr', 'OnDemand', 'Commit3Yr'], dtype=object)

In [ ]:
# Informações sobre a localização (região) do serviço
df_gcp_skus['region'] = df_gcp_skus['serviceRegions'].apply(lambda x: x[0])
df_gcp_skus.drop(columns=['serviceRegions', 'geoTaxonomy'], inplace=True)
df_gcp_skus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24649 entries, 1 to 28078
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 24649 non-null  object
 1   skuId                24649 non-null  object
 2   description          24649 non-null  object
 3   pricingInfo          24649 non-null  object
 4   serviceProviderName  24649 non-null  object
 5   serviceName          24649 non-null  object
 6   resourceType         24649 non-null  object
 7   resourceName         24649 non-null  object
 8   usage                24649 non-null  object
 9   region               24649 non-null  object
dtypes: object(10)
memory usage: 2.1+ MB


/tmp/ipython-input-24-3738931337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp_skus['region'] = df_gcp_skus['serviceRegions'].apply(lambda x: x[0])
/tmp/ipython-input-24-3738931337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp_skus.drop(columns=['serviceRegions', 'geoTaxonomy'], inplace=True)


In [ ]:
# Eliminando registros de skus cujo localização não será objeto de análise na presente investigação
df_gcp_skus = df_gcp_skus.loc[df_gcp_skus['region'] != 'global']

In [ ]:
# Informações sobre custo por hora - Definição da Função
def extrair_preco_horario(pricing_info):
    try:
        # Acessa o primeiro item da lista 'pricingInfo'
        rates = pricing_info[0]['pricingExpression']['tieredRates']
        # Acessa o primeiro nível de preço na lista 'tieredRates'
        unit_price = rates[0]['unitPrice']

        # Pega os valores de 'units' e 'nanos'. Usa .get(key, 0) para retornar 0 se a chave não existir.
        units = int(unit_price.get('units', 0))
        nanos = int(unit_price.get('nanos', 0))

        # Retorna o preço final calculado
        return (units + nanos / 1_000_000_000)
    except (TypeError, IndexError, KeyError):
        # Se qualquer parte da estrutura estiver faltando ou for diferente, retorna Nulo
        return None

In [ ]:
# Informações sobre custo por hora - Aplicação da Função
df_gcp_skus['hourPrice'] = df_gcp_skus['pricingInfo'].apply(extrair_preco_horario)
df_gcp_skus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23488 entries, 1 to 28078
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 23488 non-null  object 
 1   skuId                23488 non-null  object 
 2   description          23488 non-null  object 
 3   pricingInfo          23488 non-null  object 
 4   serviceProviderName  23488 non-null  object 
 5   serviceName          23488 non-null  object 
 6   resourceType         23488 non-null  object 
 7   resourceName         23488 non-null  object 
 8   usage                23488 non-null  object 
 9   region               23488 non-null  object 
 10  hourPrice            23487 non-null  float64
dtypes: float64(1), object(10)
memory usage: 2.2+ MB


/tmp/ipython-input-27-2710857582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp_skus['hourPrice'] = df_gcp_skus['pricingInfo'].apply(extrair_preco_horario)


### Estruturando o Dataframe

In [ ]:
print("--- Bloco 1: Preparando os Catálogos de Preços (On-Demand e Reserva) ---")

# 1. Isola os preços de CPU para o modelo On-Demand
# CORREÇÃO: Usamos o campo 'category.usageType' para um filtro preciso.
df_cpu_ondemand = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'CPU') &
    (df_gcp_skus['usage'] == 'OnDemand')
].copy()

# 2. Isola os preços de RAM para o modelo On-Demand
# CORREÇÃO: Usamos o campo 'category.usageType' aqui também.
df_ram_ondemand = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'RAM') &
    (df_gcp_skus['usage'] == 'OnDemand')
].copy()

# CUDs de 1 Ano
df_cpu_1yr = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'CPU') &
    (df_gcp_skus['usage'] == 'Commit1Yr')
].copy()

df_ram_1yr = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'RAM') &
    (df_gcp_skus['usage'] == 'Commit1Yr')
].copy()

# CUDs de 3 Anos
df_cpu_3yr = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'CPU') &
    (df_gcp_skus['usage'] == 'Commit3Yr')
].copy()

df_ram_3yr = df_gcp_skus[
    (df_gcp_skus['resourceName'] == 'RAM') &
    (df_gcp_skus['usage'] == 'Commit3Yr')
].copy()

print("Catálogos de preços para On-Demand, CUD 1 Ano e CUD 3 Anos foram criados.")
print(f"Total de SKUs de CPU On-Demand: {len(df_cpu_ondemand)}")
print(f"Total de SKUs de CPU CUD 1 Ano: {len(df_cpu_1yr)}")
print(f"Total de SKUs de CPU CUD 3 Anos: {len(df_cpu_3yr)}")
print(f"Total de SKUs de RAM On-Demand: {len(df_ram_ondemand)}")
print(f"Total de SKUs de RAM CUD 1 Ano: {len(df_ram_1yr)}")
print(f"Total de SKUs de RAM CUD 3 Anos: {len(df_ram_3yr)}")

--- Bloco 1: Preparando os Catálogos de Preços (On-Demand e Reserva) ---
Catálogos de preços para On-Demand, CUD 1 Ano e CUD 3 Anos foram criados.
Total de SKUs de CPU On-Demand: 2743
Total de SKUs de CPU CUD 1 Ano: 827
Total de SKUs de CPU CUD 3 Anos: 827
Total de SKUs de RAM On-Demand: 2894
Total de SKUs de RAM CUD 1 Ano: 827
Total de SKUs de RAM CUD 3 Anos: 827


In [ ]:
# Armazena os dataframes de preços em dicionários para fácil acesso
cpu_prices = {'OnDemand': df_cpu_ondemand, '1yr': df_cpu_1yr, '3yr': df_cpu_3yr}
ram_prices = {'OnDemand': df_ram_ondemand, '1yr': df_ram_1yr, '3yr': df_ram_3yr}

In [ ]:
for _, row in df_gcp_skus.loc[df_gcp_skus['resourceName'] == 'CPU'].head().iterrows():
    print(row['description'])

Commitment v1: Memory-optimized Cpu in Phoenix for 1 Year
Commitment v1: Cpu in Montreal for 1 Year
Sole Tenancy Premium for N2D AMD Sole Tenancy Instance Core running in Columbus
DWS Defined Duration G2 Core running in Alabama
Commitment v1: C3D Cpu in Paris for 1 Year


In [ ]:
# --- Bloco 2: Função de Cálculo Final ---
def calcular_preco_vm_gcp(row, tipo_cobranca='OnDemand'):
    """
    Calcula o preço de uma VM para um tipo de cobrança específico (OnDemand, 1yr, 3yr).
    """
    familia = row['machineFamily']
    regiao = row['regionCode']
    num_cpus = row['numberOfCores']
    ram_gb = row['memoryInMiB'] / 1024

    # Seleciona o catálogo de preços correto com base no tipo_cobranca
    df_cpu = cpu_prices.get(tipo_cobranca)
    df_ram = ram_prices.get(tipo_cobranca)

    if df_cpu is None or df_ram is None:
        return None # Tipo de cobrança inválido

    # --- Busca Preço da CPU ---
    # O padrão de busca muda ligeiramente para CUDs
    filtro_cpu = (df_cpu['description'].str.contains(familia, na=False)) & \
                 (df_cpu['region'].apply(lambda x: regiao in x))
    preco_cpu_val = df_cpu.loc[filtro_cpu, 'hourPrice'].values
    preco_cpu = preco_cpu_val[0] if len(preco_cpu_val) > 0 else None

    # --- Busca Preço da RAM ---
    filtro_ram = (df_ram['description'].str.contains(familia, na=False)) & \
                 (df_ram['region'].apply(lambda x: regiao in x))
    preco_ram_val = df_ram.loc[filtro_ram, 'hourPrice'].values
    preco_ram = preco_ram_val[0] if len(preco_ram_val) > 0 else None

    # --- Calcula o Preço ---
    if preco_cpu is not None and preco_ram is not None:
        preco_hardware = (num_cpus * preco_cpu) + (ram_gb * preco_ram)
        return preco_hardware
    else:
        return None

In [ ]:
for _, row in df_gcp_specs.tail().iterrows():
    print(calcular_preco_vm_gcp(row))

1.52256
0.19032
2.28384
2.8548
0.38064


In [ ]:
df_gcp_specs['priceOnDemand'] = df_gcp_specs.apply(calcular_preco_vm_gcp, axis=1)

In [ ]:
df_gcp_specs['priceReserved1yr'] = df_gcp_specs.apply(calcular_preco_vm_gcp, axis=1, tipo_cobranca='1yr')

In [ ]:
df_gcp_specs['priceReserved3yr'] = df_gcp_specs.apply(calcular_preco_vm_gcp, axis=1, tipo_cobranca='3yr')

In [ ]:
df_gcp_specs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25365 entries, 0 to 25364
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   machineType       25365 non-null  object 
 1   machineFamily     25365 non-null  object 
 2   numberOfCores     25365 non-null  int64  
 3   memoryInMiB       25365 non-null  int64  
 4   ephemeralDiskGiB  25365 non-null  int64  
 5   regionCode        25365 non-null  object 
 6   priceOnDemand     20549 non-null  float64
 7   priceReserved1yr  20635 non-null  float64
 8   priceReserved3yr  20635 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.7+ MB


In [ ]:
general_purpose = ['N4', 'N2', 'N2D', 'N1', 'C4A', 'C4', 'C4D', 'C3', 'C3D', 'E2', 'T2A', 'T2D']
df_vms_gcp = df_gcp_specs.loc[
    (df_gcp_specs['priceOnDemand'].notna()) &
    (df_gcp_specs['machineFamily'].isin(general_purpose))
    ]

df_vms_gcp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17744 entries, 1109 to 25364
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   machineType       17744 non-null  object 
 1   machineFamily     17744 non-null  object 
 2   numberOfCores     17744 non-null  int64  
 3   memoryInMiB       17744 non-null  int64  
 4   ephemeralDiskGiB  17744 non-null  int64  
 5   regionCode        17744 non-null  object 
 6   priceOnDemand     17744 non-null  float64
 7   priceReserved1yr  17688 non-null  float64
 8   priceReserved3yr  17688 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.4+ MB


## Tratamento 💆

In [ ]:
df_vms_gcp['extractionDate'] = extraction_date
df_vms_gcp['provider'] = 'GCP'
df_vms_gcp['currencyCode'] = 'USD'

/tmp/ipython-input-39-4161156711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vms_gcp['extractionDate'] = extraction_date
/tmp/ipython-input-39-4161156711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vms_gcp['provider'] = 'GCP'
/tmp/ipython-input-39-4161156711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [ ]:
# Este mapa foi criado com base na documentação oficial do GCP.
mapa_regioes_gcp = {
    'asia-east1': 'Taiwan',
    'asia-east2': 'Hong Kong',
    'asia-northeast1': 'Tokyo, Japan',
    'asia-northeast2': 'Osaka, Japan',
    'asia-northeast3': 'Seoul, South Korea',
    'asia-south1': 'Mumbai, India',
    'asia-south2': 'Delhi, India',
    'asia-southeast1': 'Singapore',
    'asia-southeast2': 'Jakarta, Indonesia',
    'australia-southeast1': 'Sydney, Australia',
    'australia-southeast2': 'Melbourne, Australia',
    'europe-central2': 'Warsaw, Poland',
    'europe-north1': 'Finland',
    'europe-west1': 'Belgium',
    'europe-west2': 'London, England, UK',
    'europe-west3': 'Frankfurt, Germany',
    'europe-west4': 'Netherlands',
    'europe-west6': 'Zurich, Switzerland',
    'northamerica-northeast1': 'Montréal, Canada',
    'northamerica-northeast2': 'Toronto, Canada',
    'southamerica-east1': 'São Paulo, Brazil',
    'southamerica-west1': 'Santiago, Chile',
    'us-central1': 'Iowa, USA',
    'us-east1': 'South Carolina, USA',
    'us-east4': 'Northern Virginia, USA',
    'us-west1': 'Oregon, USA',
    'us-west2': 'Los Angeles, USA',
    'us-west3': 'Salt Lake City, USA',
    'us-west4': 'Las Vegas, USA',
    'me-west1': 'Tel Aviv, Israel',
    'africa-south1': 'Johannesburg, South Africa'
    # Adicione outras regiões conforme necessário
}

# Usa o dicionário para mapear os nomes programáticos para os nomes legíveis.
# .get(x, x) é usado como um fallback: se uma região não estiver no mapa, ele usa o próprio nome programático.
df_vms_gcp['region'] = df_vms_gcp['regionCode'].apply(lambda x: mapa_regioes_gcp.get(x, x))
print("Coluna 'region' criada com sucesso.")

# Exibe o resultado para verificação
display(df_vms_gcp[['regionCode', 'region']].head())

Coluna 'region' criada com sucesso.


/tmp/ipython-input-40-1332737139.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vms_gcp['region'] = df_vms_gcp['regionCode'].apply(lambda x: mapa_regioes_gcp.get(x, x))


,regionCode,region
1109,us-east4,"Northern Virginia, USA"
1110,us-east4,"Northern Virginia, USA"
1111,us-east4,"Northern Virginia, USA"
1112,us-east4,"Northern Virginia, USA"
1113,us-east4,"Northern Virginia, USA"


In [ ]:
ordered_columns = ['extractionDate', 'provider', 'machineType', 'regionCode', 'region', 'numberOfCores', 'memoryInMiB', 'currencyCode', 'priceOnDemand', 'priceReserved1yr', 'priceReserved3yr']
df_vms_gcp = df_vms_gcp[ordered_columns]
df_vms_gcp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17744 entries, 1109 to 25364
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype                            
---  ------            --------------  -----                            
 0   extractionDate    17744 non-null  datetime64[us, America/Sao_Paulo]
 1   provider          17744 non-null  object                           
 2   machineType       17744 non-null  object                           
 3   regionCode        17744 non-null  object                           
 4   region            17744 non-null  object                           
 5   numberOfCores     17744 non-null  int64                            
 6   memoryInMiB       17744 non-null  int64                            
 7   currencyCode      17744 non-null  object                           
 8   priceOnDemand     17744 non-null  float64                          
 9   priceReserved1yr  17688 non-null  float64                          
 10  priceReserve

## Persistência 📁

In [ ]:
file_name = f'vms_gcp_{extraction_date.strftime("%d-%m-%Y")}.csv'
set_key(env_file, "GCP_FILE", file_name)

df_vms_gcp.to_csv(f'/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Dados/{file_name}', index=False)